# Estimate Tests

- Create the internal consistency code
- build the diff file code for estimates 

In [1]:
# Import Libraries
import pandas as pd
import numpy as np
import yaml
import os
import pyodbc
import copy
import pandas as pd
import pyodbc
import os
import glob
import numpy as np

What internal consistency will do 


Functions: 
1. Function where you pass in the three levels and it checks for equivalence between them and outputs differences, maybe a helpful dataframe 
2. Total Population for the region, jurisdiction, cpa matches the sum of household population and GQ population 
- Could output a dataframe where all the sums are there then filter for greater than zero, or output the whole dataframe 
3. Total Population for the region, jurisdiction, cpa matches the sum of all ethnicity group population 
4. Total Population for the region, jurisdiction, cpa matches the sum of all age-group population 
5. Total Population for the region, jurisdiction, cpa matches the sum of male and female population from gender table 

In [2]:
def internal_consistency():
    '''
    Along with an individual check for each column sum 

    Total Population for the region, jurisdiction, cpa matches the sum of household population and GQ population 

Total Population for the region, jurisdiction, cpa matches the sum of all ethnicity group population 

Total Population for the region, jurisdiction, cpa matches the sum of all age-group population 

Total Population for the region, jurisdiction, cpa matches the sum of male and female population from gender table 
    
    '''

# Internal Consistency

In [2]:

# importing the required modules
import glob
import pandas as pd
  
# specifying the path to csv files
path = "C:/Users/cra/San Diego Association of Governments/SANDAG QA QC - Documents/Projects/2022/2022-56 Estimates 2021 QC/Data/2021_01/Raw Data"
  
# csv files in the path
files = glob.glob(path + "/*.csv")

new = {}
for path in files:
    df = pd.read_csv(path)
    if 'Unnamed: 0' in df.columns:
        df = df.drop('Unnamed: 0', axis = 1)
        print('Has')
    new[path.split('\\')[1][:-4]] = df

In [3]:
new.keys()

dict_keys(['2021_01_age_cpa_QA', '2021_01_age_ethnicity_cpa_QA', '2021_01_age_ethnicity_jurisdiction_QA', '2021_01_age_ethnicity_region_QA', '2021_01_age_jurisdiction_QA', '2021_01_age_region_QA', '2021_01_age_sex_ethnicity_cpa_QA', '2021_01_age_sex_ethnicity_jurisdiction_QA', '2021_01_age_sex_ethnicity_region_QA', '2021_01_consolidated_cpa_QA', '2021_01_consolidated_jurisdiction_QA', '2021_01_consolidated_region_QA', '2021_01_ethnicity_cpa_QA', '2021_01_ethnicity_jurisdiction_QA', '2021_01_ethnicity_region_QA', '2021_01_households_cpa_QA', '2021_01_households_jurisdiction_QA', '2021_01_households_region_QA', '2021_01_household_income_cpa_QA', '2021_01_household_income_jurisdiction_QA', '2021_01_household_income_region_QA'])

In [3]:

# importing the required modules
import glob
import pandas as pd
  
# specifying the path to csv files
path = "C:/Users/cra/San Diego Association of Governments/SANDAG QA QC - Documents/Projects/2022/2022-56 Estimates 2021 QC/Data/2020_06/Raw Data"
  
# csv files in the path
files = glob.glob(path + "/*.csv")

old = {}
for path in files:
    old[path.split('\\')[1][:-4]] = pd.read_csv(path)

In [7]:
old.keys()

dict_keys(['2020_06_age_cpa_QA', '2020_06_age_ethnicity_cpa_QA', '2020_06_age_ethnicity_jurisdiction_QA', '2020_06_age_ethnicity_region_QA', '2020_06_age_jurisdiction_QA', '2020_06_age_region_QA', '2020_06_age_sex_ethnicity_cpa_QA', '2020_06_age_sex_ethnicity_jurisdiction_QA', '2020_06_age_sex_ethnicity_region_QA', '2020_06_consolidated_cpa_QA', '2020_06_consolidated_jurisdiction_QA', '2020_06_consolidated_region_QA', '2020_06_ethnicity_cpa_QA', '2020_06_ethnicity_jurisdiction_QA', '2020_06_ethnicity_region_QA', '2020_06_household_income_cpa_QA', '2020_06_household_income_jurisdiction_QA', '2020_06_household_income_region_QA'])

### Compare Totals 
This is comparing the totals accross CPA, Jur, and Region to see if they are equivalent. 

In [4]:
def compare_totals(cpa_df, jur_df, region_df):
    """
    Aggregates sum of values by year in cpa, jurisdiction, and region level and compares them together to see if they match. CPA is removed as many areas calculated in the other georgraphy 
    """
    cpa_totals = cpa_df.groupby('yr_id').sum()
    jur_totals = jur_df.groupby('yr_id').sum()
    reg_totals = region_df
    
    non_matches_dict = {}
    jur_non_matches = []
    for col in cpa_totals.columns:
        if not np.isclose(jur_totals[col], cpa_totals[col]).all():
            jur_non_matches.append(col)
                        
    if len(jur_non_matches) == len(cpa_totals.columns):
        non_matches_dict['jurisdiction'] = 'all columns did not match.'
    elif len(jur_non_matches) == 0:
        non_matches_dict['jurisdiction'] = 'all columns matched.'
    else:
        if len(jur_non_matches) > 10:
            non_matches_dict['jurisdiction'] = f"{len(jur_non_matches)} columns did not match out of {len(cpa_totals.columns)} columns."
        else:
            non_matches_dict['jurisdiction'] = ', '.join(jur_non_matches) + ' columns did not match.'
    
    reg_non_matches = []
    for col in cpa_totals.columns:
        if not np.isclose(reg_totals[col], cpa_totals[col]).all():
            reg_non_matches.append(col)
    if len(reg_non_matches) == len(cpa_totals.columns):
        non_matches_dict['region'] = 'all columns did not match.'
    elif len(reg_non_matches) == 0:
        non_matches_dict['region'] = 'all columns matched.'
    else:
        if len(reg_non_matches) > 10:
            non_matches_dict['region'] = f"{len(reg_non_matches)} columns did not match out of {len(cpa_totals.columns)}."
        else:
            non_matches_dict['region'] = ', '.join(reg_non_matches) + ' columns did not match.'
    
    return non_matches_dict

In [5]:
compare_totals(new['2021_01_consolidated_cpa_QA'], new['2021_01_consolidated_jurisdiction_QA'], new['2021_01_consolidated_region_QA'])

{'jurisdiction': 'all columns matched.', 'region': 'all columns matched.'}

### Ethnicity population check 
Total Population for the region, jurisdiction, cpa matches the sum of all ethnicity group population 

In [6]:
def ethnicity_internal_consistency(cpa_df, jur_df, region_df):
    cpa_totals = cpa_df.groupby('yr_id').sum()
    jur_totals = jur_df.groupby('yr_id').sum()
    reg_totals = region_df


    results = {}

    for geography_data in [('CPA',cpa_totals), ('Jurisdcition',jur_totals), ('Region',reg_totals)]:
        
        geo_level = geography_data[0]
        df = geography_data[1]

        pop_test = df['Hispanic'] + df['Non-Hispanic, White'] + df['Non-Hispanic, Black'] + df['Non-Hispanic, American Indian or Alaska Native'] + df['Non-Hispanic, Asian'] + df['Non-Hispanic, Hawaiian or Pacific Islander'] + df['Non-Hispanic, Other'] + df['Non-Hispanic, Two or More Races']

        if sum(~(pop_test == df['Total Population'])) != 0:
            results[geo_level] = 'Ethnicity population does not sum up correctly'
        else:
            results[geo_level] = 'Pass'
    
    return results


In [7]:
ethnicity_internal_consistency(new['2021_01_consolidated_cpa_QA'], new['2021_01_consolidated_jurisdiction_QA'], new['2021_01_consolidated_region_QA'])

{'CPA': 'Pass', 'Jurisdcition': 'Pass', 'Region': 'Pass'}

### Age Group Comparison Check
Total Population for the region, jurisdiction, cpa matches the sum of all age-group population 

In [10]:
def age_group_internal_consistency(cpa_df, jur_df, region_df):
    cpa_totals = cpa_df.groupby('yr_id').sum()
    jur_totals = jur_df.groupby('yr_id').sum()
    reg_totals = region_df


    results = {}

    for geography_data in [('CPA',cpa_totals), ('Jurisdcition',jur_totals), ('Region',reg_totals)]:
        
        geo_level = geography_data[0]
        df = geography_data[1]

        pop_test = df['Under 5'] + df['5 to 9'] + df['10 to 14'] + df['15 to 17'] + df['18 and 19'] + df['20 to 24'] + df['25 to 29'] + df['30 to 34'] + df['35 to 39'] + df['40 to 44'] + df['45 to 49'] + df['50 to 54'] + df['55 to 59'] + df['60 and 61'] + df['62 to 64'] + df['65 to 69'] + df['70 to 74'] + df['75 to 79'] + df['80 to 84'] + df['85 and Older']

        if sum(~(pop_test == df['Total Population'])) != 0:
            results[geo_level] = 'Ethnicity population does not sum up correctly'
        else:
            results[geo_level] = 'Pass'
    
    return results

In [11]:
age_group_internal_consistency(new['2021_01_consolidated_cpa_QA'], new['2021_01_consolidated_jurisdiction_QA'], new['2021_01_consolidated_region_QA'])

{'CPA': 'Pass', 'Jurisdcition': 'Pass', 'Region': 'Pass'}

### Gender Population test 
Total Population for the region, jurisdiction, cpa matches the sum of male and female population from gender table 

In [12]:
def gender_internal_consistency(cpa_df, jur_df, region_df):
    cpa_totals = cpa_df.groupby('yr_id').sum()
    jur_totals = jur_df.groupby('yr_id').sum()
    reg_totals = region_df


    results = {}

    for geography_data in [('CPA',cpa_totals), ('Jurisdcition',jur_totals), ('Region',reg_totals)]:
        
        geo_level = geography_data[0]
        df = geography_data[1]

        pop_test = df['Female'] + df['Male']

        if sum(~(pop_test == df['Total Population'])) != 0:
            results[geo_level] = 'Ethnicity population does not sum up correctly'
        else:
            results[geo_level] = 'Pass'
    
    return results

In [13]:
gender_internal_consistency(new['2021_01_consolidated_cpa_QA'], new['2021_01_consolidated_jurisdiction_QA'], new['2021_01_consolidated_region_QA'])

{'CPA': 'Pass', 'Jurisdcition': 'Pass', 'Region': 'Pass'}

# Checking for Null Values
Code pulled from Prabina's code with slight modifications.

In [18]:
def spot_nulls(df):
    if (df.isna().sum() != 0).all():
        print('Null values present')
        return df.columns[df.isnull().any()].tolist() # returns the column names that have null vals
    else:
        print('No nulls Present')

In [19]:
spot_nulls(new['2021_01_consolidated_cpa_QA'])

No nulls Present


In [20]:
new.keys()

dict_keys(['2021_01_age_cpa_QA', '2021_01_age_ethnicity_cpa_QA', '2021_01_age_ethnicity_jurisdiction_QA', '2021_01_age_ethnicity_region_QA', '2021_01_age_jurisdiction_QA', '2021_01_age_region_QA', '2021_01_age_sex_ethnicity_cpa_QA', '2021_01_age_sex_ethnicity_jurisdiction_QA', '2021_01_age_sex_ethnicity_region_QA', '2021_01_consolidated_cpa_QA', '2021_01_consolidated_jurisdiction_QA', '2021_01_consolidated_region_QA', '2021_01_ethnicity_cpa_QA', '2021_01_ethnicity_jurisdiction_QA', '2021_01_ethnicity_region_QA', '2021_01_households_cpa_QA', '2021_01_households_jurisdiction_QA', '2021_01_households_region_QA', '2021_01_household_income_cpa_QA', '2021_01_household_income_jurisdiction_QA', '2021_01_household_income_region_QA'])

In [21]:
spot_nulls(new['2021_01_age_sex_ethnicity_region_QA'])

No nulls Present


# Creating Diff Files

In [29]:
def diff_file(df1, df2, geo_level=None):
    '''
    This function creates a diff file of two dataframes with identical labels and identical shapes and creates a diff file of the two. 
    This does df1 minus df2
    
    '''
    if sum(~(df1.columns == df2.columns)) != 0:
        print('Columns are not identical, cannot preform diff')


    if geo_level == None:
        print('Please input a geography level')
    elif geo_level == 'cpa':
        new_df = (df1.set_index(['cpa', 'yr_id']) - df2.set_index(['cpa', 'yr_id'])).reset_index()    
    elif geo_level == 'jurisdiction':
        new_df = (df1.set_index(['jurisdiction', 'yr_id']) - df2.set_index(['jurisdiction', 'yr_id'])).reset_index()
    elif geo_level == 'region':
        new_df = (df1.set_index(['region', 'yr_id']) - df2.set_index(['region', 'yr_id'])).reset_index()

    
    return new_df

In [28]:
# Special for age_sex_ethnicity dataframe
def diff_file(df1, df2, geo_level=None):
    '''
    This function creates a diff file of two dataframes with identical labels and identical shapes and creates a diff file of the two. 
    This does df1 minus df2
    
    '''
    if sum(~(df1.columns == df2.columns)) != 0:
        print('Columns are not identical, cannot preform diff')


    if geo_level == None:
        print('Please input a geography level')
    elif geo_level == 'cpa':
        new_df = (df1.set_index(['cpa', 'yr_id', 'name', 'sex']) - df2.set_index(['cpa', 'yr_id', 'name', 'sex'])).reset_index()    
    elif geo_level == 'jurisdiction':
        new_df = (df1.set_index(['jurisdiction', 'yr_id', 'name', 'sex']) - df2.set_index(['jurisdiction', 'yr_id', 'name', 'sex'])).reset_index()
    elif geo_level == 'region':
        new_df = (df1.set_index(['region', 'yr_id', 'name', 'sex']) - df2.set_index(['region', 'yr_id', 'name', 'sex'])).reset_index()

    
    return new_df

In [7]:
new.keys()

dict_keys(['2021_01_age_cpa_QA', '2021_01_age_ethnicity_cpa_QA', '2021_01_age_ethnicity_jurisdiction_QA', '2021_01_age_ethnicity_region_QA', '2021_01_age_jurisdiction_QA', '2021_01_age_region_QA', '2021_01_age_sex_ethnicity_cpa_QA', '2021_01_age_sex_ethnicity_jurisdiction_QA', '2021_01_age_sex_ethnicity_region_QA', '2021_01_consolidated_cpa_QA', '2021_01_consolidated_jurisdiction_QA', '2021_01_consolidated_region_QA', '2021_01_ethnicity_cpa_QA', '2021_01_ethnicity_jurisdiction_QA', '2021_01_ethnicity_region_QA', '2021_01_households_cpa_QA', '2021_01_households_jurisdiction_QA', '2021_01_households_region_QA', '2021_01_household_income_cpa_QA', '2021_01_household_income_jurisdiction_QA', '2021_01_household_income_region_QA'])

In [8]:
old.keys()

dict_keys(['2020_06_age_cpa_QA', '2020_06_age_ethnicity_cpa_QA', '2020_06_age_ethnicity_jurisdiction_QA', '2020_06_age_ethnicity_region_QA', '2020_06_age_jurisdiction_QA', '2020_06_age_region_QA', '2020_06_age_sex_ethnicity_cpa_QA', '2020_06_age_sex_ethnicity_jurisdiction_QA', '2020_06_age_sex_ethnicity_region_QA', '2020_06_consolidated_cpa_QA', '2020_06_consolidated_jurisdiction_QA', '2020_06_consolidated_region_QA', '2020_06_ethnicity_cpa_QA', '2020_06_ethnicity_jurisdiction_QA', '2020_06_ethnicity_region_QA', '2020_06_household_income_cpa_QA', '2020_06_household_income_jurisdiction_QA', '2020_06_household_income_region_QA'])

In [60]:
# Col wise
data_dof = pd.read_csv('C:/Users/cra/San Diego Association of Governments/SANDAG QA QC - Documents/Projects/2022/2022-56 Estimates 2021 QC/Data/2021_01/Proportion Checks/age_sex_ethnicity_col_wise_sums.csv')
col_dof = pd.read_csv('C:/Users/cra/San Diego Association of Governments/SANDAG QA QC - Documents/Projects/2022/2022-56 Estimates 2021 QC/Data/2021_01/DOF_Data/DOF_age_sex_ethnicity_col_wise_sums.csv')

In [61]:
data_dof = data_dof[['region', 'yr_id', 'name', 'sex', 'Hispanic', 'Non-Hispanic, White',
       'Non-Hispanic, Asian', 'Non-Hispanic, Hawaiian or Pacific Islander',
       'Non-Hispanic, American Indian or Alaska Native',
       'Non-Hispanic, Two or More Races', 'Non-Hispanic, Black']]# removed non hispanic other

In [62]:
data_dof.shape

(480, 11)

In [63]:
data_dof['sex'] = col_dof['sex']

In [64]:
col_dof.shape

(480, 11)

In [65]:
col_dof = col_dof[['region', 'yr_id', 'name', 'sex', 'Hispanic', 'Non-Hispanic, White',
       'Non-Hispanic, Asian', 'Non-Hispanic, Hawaiian or Pacific Islander',
       'Non-Hispanic, American Indian or Alaska Native',
       'Non-Hispanic, Two or More Races', 'Non-Hispanic, Black']]

In [66]:
table = diff_file(data_dof, col_dof, geo_level='region')
table

,region,yr_id,name,sex,Hispanic,"Non-Hispanic, White","Non-Hispanic, Asian","Non-Hispanic, Hawaiian or Pacific Islander","Non-Hispanic, American Indian or Alaska Native","Non-Hispanic, Two or More Races","Non-Hispanic, Black"
0,San Diego,2010,10 to 14,F,0.021561,0.002368,-0.009153,0.068970,-0.035449,-0.078479,-0.015411
1,San Diego,2010,10 to 14,M,0.007028,-0.004973,-0.014111,0.059427,-0.162044,-0.124233,0.004538
2,San Diego,2010,15 to 17,F,0.295709,-0.124931,-0.301220,-0.169267,-0.086792,0.353501,0.179717
3,San Diego,2010,15 to 17,M,0.328962,-0.166806,-0.247008,-0.099522,0.027308,0.383080,0.180005
4,San Diego,2010,18 and 19,F,0.152597,-0.064948,0.038031,0.094088,-0.212420,-0.085410,-0.052656
...,...,...,...,...,...,...,...,...,...,...,...
475,San Diego,2021,80 to 84,M,-0.002961,-0.017871,-0.026333,0.054404,-0.027302,0.003168,0.008439
476,San Diego,2021,85 and Older,F,-0.016035,-0.013002,-0.034302,-0.024285,0.079365,0.051725,-0.016126
477,San Diego,2021,85 and Older,M,-0.007469,-0.014931,-0.010849,-0.021384,0.049011,0.028539,-0.012501
478,San Diego,2021,Under 5,F,0.131091,0.042229,0.055036,0.032791,-0.028686,-0.006429,0.036967


In [67]:
with pd.ExcelWriter("C:/Users/cra/San Diego Association of Governments/SANDAG QA QC - Documents/Projects/2022/2022-56 Estimates 2021 QC/Data/2021_01/Diff Files/DOF_diff_col_wise.xlsx") as writer:
   
    # use to_excel function and specify the sheet_name and index
    # to store the dataframe in specified sheet
    data_dof.to_excel(writer, sheet_name="DOF", index=False)
    col_dof.to_excel(writer, sheet_name="2021_01", index=False)
    table.to_excel(writer, sheet_name="Diff - DOF minus 2021_01", index=False)

In [84]:
# Row Wise analysis 
data_dof = pd.read_csv('C:/Users/cra/San Diego Association of Governments/SANDAG QA QC - Documents/Projects/2022/2022-56 Estimates 2021 QC/Data/2021_01/Proportion Checks/age_sex_ethnicity_row_wise_sums.csv')
col_dof = pd.read_csv('C:/Users/cra/San Diego Association of Governments/SANDAG QA QC - Documents/Projects/2022/2022-56 Estimates 2021 QC/Data/2021_01/DOF_Data/DOF_age_sex_ethnicity_row_wise_sums.csv')

In [85]:
data_dof.shape

(480, 12)

In [86]:
col_dof.shape

(480, 11)

In [87]:
data_dof = data_dof[['region', 'yr_id', 'name', 'sex', 'Hispanic', 'Non-Hispanic, White',
       'Non-Hispanic, Asian', 'Non-Hispanic, Hawaiian or Pacific Islander',
       'Non-Hispanic, American Indian or Alaska Native',
       'Non-Hispanic, Two or More Races', 'Non-Hispanic, Black']]# removed non hispanic other
data_dof['sex'] = col_dof['sex']

In [88]:
col_dof = col_dof[['region', 'yr_id', 'name', 'sex', 'Hispanic', 'Non-Hispanic, White',
       'Non-Hispanic, Asian', 'Non-Hispanic, Hawaiian or Pacific Islander',
       'Non-Hispanic, American Indian or Alaska Native',
       'Non-Hispanic, Two or More Races', 'Non-Hispanic, Black']]

In [89]:
table = diff_file(data_dof, col_dof, geo_level='region')
table

,region,yr_id,name,sex,Hispanic,"Non-Hispanic, White","Non-Hispanic, Asian","Non-Hispanic, Hawaiian or Pacific Islander","Non-Hispanic, American Indian or Alaska Native","Non-Hispanic, Two or More Races","Non-Hispanic, Black"
0,San Diego,2010,10 to 14,F,0.003099,-0.172497,-0.130148,0.004132,-0.010329,0.127049,-0.083666
1,San Diego,2010,10 to 14,M,-0.006684,-0.177139,-0.118301,0.003281,-0.026926,0.099129,-0.038318
2,San Diego,2010,15 to 17,F,4.222597,-3.591888,-1.801953,-0.047799,-0.030094,0.668915,0.327214
3,San Diego,2010,15 to 17,M,4.447925,-4.271212,-1.398805,-0.030476,-0.003211,0.673394,0.299005
4,San Diego,2010,18 and 19,F,2.831824,-2.734724,0.067719,0.018460,-0.077871,-0.053076,-0.280565
...,...,...,...,...,...,...,...,...,...,...,...
475,San Diego,2021,80 to 84,M,0.094490,-0.134590,-0.242852,0.032685,-0.011220,0.068619,0.071840
476,San Diego,2021,85 and Older,F,-0.244522,0.225101,-0.213407,-0.006775,0.030432,0.182179,-0.052114
477,San Diego,2021,85 and Older,M,-0.134264,-0.010863,-0.065229,-0.009293,0.032999,0.176054,-0.062468
478,San Diego,2021,Under 5,F,0.425793,-0.462668,-0.092663,-0.007021,-0.019142,0.031116,-0.112649


In [90]:
with pd.ExcelWriter("C:/Users/cra/San Diego Association of Governments/SANDAG QA QC - Documents/Projects/2022/2022-56 Estimates 2021 QC/Data/2021_01/Diff Files/DOF_diff_row_wise.xlsx") as writer:
   
    # use to_excel function and specify the sheet_name and index
    # to store the dataframe in specified sheet
    data_dof.to_excel(writer, sheet_name="DOF", index=False)
    col_dof.to_excel(writer, sheet_name="2021_01", index=False)
    table.to_excel(writer, sheet_name="Diff - DOF minus 2021_01", index=False)

# Year on Year output file
- Is this meant to be for every file that we have? Or just the consolidated ones

In [37]:
def yoy_pct_diff(df, geo_level=None):
    '''
    This finds the percent change by year
    df must be already indexed
    
    '''
    if geo_level == None:
        print('Please input a valid geography level')
    elif geo_level == 'region':
        final = df.pct_change()*100
    elif geo_level == 'cpa':
        final = pd.DataFrame()
        for geography in list(set(df.index.get_level_values(0))):
            final = final.append(df.loc[[geography]].pct_change()*100)
    elif geo_level == 'jurisdiction':
        final = pd.DataFrame()
        for geography in list(set(df.index.get_level_values(0))):
            final = final.append(df.loc[[geography]].pct_change()*100)
    return final

In [38]:
new.keys()

dict_keys(['2021_01_age_cpa_QA', '2021_01_age_ethnicity_cpa_QA', '2021_01_age_ethnicity_jurisdiction_QA', '2021_01_age_ethnicity_region_QA', '2021_01_age_jurisdiction_QA', '2021_01_age_region_QA', '2021_01_age_sex_ethnicity_cpa_QA', '2021_01_age_sex_ethnicity_jurisdiction_QA', '2021_01_age_sex_ethnicity_region_QA', '2021_01_consolidated_cpa_QA', '2021_01_consolidated_jurisdiction_QA', '2021_01_consolidated_region_QA', '2021_01_ethnicity_cpa_QA', '2021_01_ethnicity_jurisdiction_QA', '2021_01_ethnicity_region_QA', '2021_01_households_cpa_QA', '2021_01_households_jurisdiction_QA', '2021_01_households_region_QA', '2021_01_household_income_cpa_QA', '2021_01_household_income_jurisdiction_QA', '2021_01_household_income_region_QA'])

In [39]:
old.keys()

dict_keys(['2020_06_age_cpa_QA', '2020_06_age_ethnicity_cpa_QA', '2020_06_age_ethnicity_jurisdiction_QA', '2020_06_age_ethnicity_region_QA', '2020_06_age_jurisdiction_QA', '2020_06_age_region_QA', '2020_06_age_sex_ethnicity_cpa_QA', '2020_06_age_sex_ethnicity_jurisdiction_QA', '2020_06_age_sex_ethnicity_region_QA', '2020_06_consolidated_cpa_QA', '2020_06_consolidated_jurisdiction_QA', '2020_06_consolidated_region_QA', '2020_06_ethnicity_cpa_QA', '2020_06_ethnicity_jurisdiction_QA', '2020_06_ethnicity_region_QA', '2020_06_household_income_cpa_QA', '2020_06_household_income_jurisdiction_QA', '2020_06_household_income_region_QA'])

In [49]:
yoy_pct_diff(old['2020_06_consolidated_region_QA'].set_index(['region', 'yr_id']), geo_level='region').to_csv('C:/Users/cra/San Diego Association of Governments/SANDAG QA QC - Documents/Projects/2022/2022-56 Estimates 2021 QC/Data/2020_06/yoy_pct_change/consolidated_region_change.csv')

# Proportion Check 
- Compare the proportion of groups (age/gender/ethnicity/ethnicity by gender by age in total population between DOF and Estimates at Region Level  

- To be performed on Age, Ethnicity, Ethnicity by age and gender, household income 

- Gender still needs to be outputted 


- What is category?
- Am I doing all of the proportion checks?



1. household income

2. age

3. gender (later)

4. ethnicity

5. ethnicity by age

6. ethnicity by age by gender



- Ask bryce which DOF table the current estimates use?
- Where is the dimension table for race_code, same as in demographic_warehouse?

In chat: 
- Start chat with whoever is in that group 


In [83]:
data.keys()

dict_keys(['2020_06_age_cpa_QA', '2020_06_age_ethnicity_cpa_QA', '2020_06_age_ethnicity_jurisdiction_QA', '2020_06_age_ethnicity_region_QA', '2020_06_age_jurisdiction_QA', '2020_06_age_region_QA', '2020_06_age_sex_ethnicity_cpa_QA', '2020_06_age_sex_ethnicity_jurisdiction_QA', '2020_06_age_sex_ethnicity_region_QA', '2020_06_consolidated_cpa_QA', '2020_06_consolidated_jurisdiction_QA', '2020_06_consolidated_region_QA', '2020_06_ethnicity_cpa_QA', '2020_06_ethnicity_jurisdiction_QA', '2020_06_ethnicity_region_QA', '2020_06_household_income_cpa_QA', '2020_06_household_income_jurisdiction_QA', '2020_06_household_income_region_QA'])

In [86]:
data['2020_06_age_ethnicity_region_QA'].set_index(['region', 'yr_id', 'name'])


'''
1st way: 
90k / sum(10-14 age group for all ethnicity)

2nd way
90k/ total hispanic population

'''

Hispanic  Non-Hispanic, White  \
region    yr_id name                                          
San Diego 2010  10 to 14         90039                68297   
                15 to 17         57838                44692   
                18 and 19        39688                37261   
                20 to 24         93539               119871   
                25 to 29         84428               114319   
...                                ...                  ...   
          2020  70 to 74         22879                88779   
                75 to 79         15173                59109   
                80 to 84         10510                39244   
                85 and Older     10061                44814   
                Under 5          86870                80213   

                              Non-Hispanic, Asian  \
region    yr_id name                                
San Diego 2010  10 to 14                    17950   
                15 to 17                    11065   
                18 and 19                    9922   
                20 to 24                    29194   
                25 to 29                    27797   
...                                           ...   
          2020  70 to 74                    15255   
                75 to 79                    10214   
                80 to 84                     7518   
                85 and Older                 7162   
                Under 5                     18202   

                              Non-Hispanic, Hawaiian or Pacific Islander  \
region    yr_id name                                                       
San Diego 2010  10 to 14                                             819   
                15 to 17                                             579   
                18 and 19                                            472   
                20 to 24                                            1439   
                25 to 29                                            1235   
...                                                                  ...   
          2020  70 to 74                                             506   
                75 to 79                                             257   
                80 to 84                                             280   
                85 and Older                                         230   
                Under 5                                              729   

                              Non-Hispanic, American Indian or Alaska Native  \
region    yr_id name                                                           
San Diego 2010  10 to 14                                                 871   
                15 to 17                                                 605   
                18 and 19                                                501   
                20 to 24                                                1496   
                25 to 29                                                1203   
...                                                                      ...   
          2020  70 to 74                                                 601   
                75 to 79                                                 340   
                80 to 84                                                 236   
                85 and Older                                             216   
                Under 5                                                  939   

                              Non-Hispanic, Other  \
region    yr_id name                                
San Diego 2010  10 to 14                      524   
                15 to 17                      344   
                18 and 19                     223   
                20 to 24                      615   
                25 to 29                      726   
...                                           ...   
          2020  70 to 74                      161   
             

In [84]:
data['2020_06_household_income_region_QA']
# formula value / sum(values)

,region,yr_id,"Less than $15,000","$15,000 to $29,999","$30,000 to $44,999","$45,000 to $59,999","$60,000 to $74,999","$75,000 to $99,999","$100,000 to $124,999","$125,000 to $149,999","$150,000 to $199,999","$200,000 or more"
0,San Diego,2010,117399,149059,153011,123744,111343,142638,100598,63023,64067,61984
1,San Diego,2011,117657,149510,153502,124118,111639,142889,100769,63130,64157,62050
2,San Diego,2012,134432,155319,146936,127768,109248,138737,97464,59502,63620,60916
3,San Diego,2013,131390,160423,151880,128231,106684,138648,94698,60000,68240,63338
4,San Diego,2014,128661,159289,152979,130555,105186,136495,99016,58772,66468,72729
5,San Diego,2015,124967,150445,148877,126271,105363,145985,104614,60883,69431,80806
6,San Diego,2016,115619,147924,144916,133206,110768,151444,107571,63025,69472,81639
7,San Diego,2017,105544,146182,145843,128844,112160,148686,111103,66105,78477,91317
8,San Diego,2018,100975,141923,136894,128215,111662,153338,114798,68592,83483,103620
9,San Diego,2019,105792,136944,138600,128470,108525,155468,115643,71304,88240,104390


# Proportion Checks 

In [67]:
new.keys()

dict_keys(['2021_01_age_cpa_QA', '2021_01_age_ethnicity_cpa_QA', '2021_01_age_ethnicity_jurisdiction_QA', '2021_01_age_ethnicity_region_QA', '2021_01_age_jurisdiction_QA', '2021_01_age_region_QA', '2021_01_age_sex_ethnicity_cpa_QA', '2021_01_age_sex_ethnicity_jurisdiction_QA', '2021_01_age_sex_ethnicity_region_QA', '2021_01_consolidated_cpa_QA', '2021_01_consolidated_jurisdiction_QA', '2021_01_consolidated_region_QA', '2021_01_ethnicity_cpa_QA', '2021_01_ethnicity_jurisdiction_QA', '2021_01_ethnicity_region_QA', '2021_01_households_cpa_QA', '2021_01_households_jurisdiction_QA', '2021_01_households_region_QA', '2021_01_household_income_cpa_QA', '2021_01_household_income_jurisdiction_QA', '2021_01_household_income_region_QA'])

In [170]:
new['2021_01_age_sex_ethnicity_region_QA']

,region,yr_id,name,sex,Hispanic,"Non-Hispanic, White","Non-Hispanic, Asian","Non-Hispanic, Hawaiian or Pacific Islander","Non-Hispanic, American Indian or Alaska Native","Non-Hispanic, Other","Non-Hispanic, Two or More Races","Non-Hispanic, Black"
0,San Diego,2010,10 to 14,Female,44161,33071,8634,385,398,254,5133,4777
1,San Diego,2010,10 to 14,Male,45878,35226,9316,434,473,270,5233,5073
2,San Diego,2010,15 to 17,Female,27852,21611,5315,270,291,156,2903,3260
3,San Diego,2010,15 to 17,Male,29986,23081,5750,309,314,188,3008,3706
4,San Diego,2010,18 and 19,Female,18981,16601,4888,204,216,103,1846,2290
...,...,...,...,...,...,...,...,...,...,...,...,...
475,San Diego,2021,80 to 84,Male,4092,17807,3038,114,118,32,303,936
476,San Diego,2021,85 and Older,Female,6828,28126,4905,140,134,33,436,1115
477,San Diego,2021,85 and Older,Male,3633,17308,2481,107,94,18,260,735
478,San Diego,2021,Under 5,Female,39979,37907,8645,334,425,230,4774,4657


In [171]:
df = new['2021_01_age_sex_ethnicity_region_QA'].groupby(['region', 'yr_id', 'name', 'sex']).sum()
sum_vals = list(df.sum(axis=1))
row_sum_percentage = df.div(sum_vals, axis=0)*100
row_sum_percentage

Hispanic  Non-Hispanic, White  \
region    yr_id name         sex                                      
San Diego 2010  10 to 14     Female  45.614742            34.159669   
                             Male    45.021246            34.568168   
                15 to 17     Female  45.171754            35.049791   
                             Male    45.199120            34.790932   
                18 and 19    Female  42.059430            36.785659   
...                                        ...                  ...   
          2021  80 to 84     Male    15.476551            67.348714   
                85 and Older Female  16.367428            67.420955   
                             Male    14.746712            70.254912   
                Under 5      Female  41.236295            39.099133   
                             Male    40.848106            39.466706   

                                     Non-Hispanic, Asian  \
region    yr_id name         sex                           
San Diego 2010  10 to 14     Female             8.918224   
                             Male               9.142027   
                15 to 17     Female             8.620130   
                             Male               8.667209   
                18 and 19    Female            10.831173   
...                                                  ...   
          2021  80 to 84     Male              11.490166   
                85 and Older Female            11.757797   
                             Male              10.070628   
                Under 5      Female             8.916876   
                             Male               8.775014   

                                     Non-Hispanic, Hawaiian or Pacific Islander  \
region    yr_id name         sex                                                  
San Diego 2010  10 to 14     Female                                    0.397674   
                             Male                                      0.425895   
                15 to 17     Female                                    0.437899   
                             Male                                      0.465768   
                18 and 19    Female                                    0.452037   
...                                                                         ...   
          2021  80 to 84     Male                                      0.431165   
                85 and Older Female                                    0.335595   
                             Male                                      0.434324   
                Under 5      Female                                    0.344504   
                             Male                                      0.355529   

                                     Non-Hispanic, American Indian or Alaska Native  \
region    yr_id name         sex                                                      
San Diego 2010  10 to 14     Female                                        0.411102   
                             Male                                          0.464167   
                15 to 17     Female                                        0.471958   
                             Male                                          0.473305   
                18 and 19    Female                                        0.478628   
...                                                                             ...   
          2021  80 to 84     Male                                          0.446293   
                85 and Older Female                                        0.321212   
                             Male                                          0.381555   
                Under 5      Female                                        0.438366   
                             Male                                          0.514425   

                                     Non-Hispanic, Other  \
region    yr_id name         sex                           
S

In [ ]:
df = new['2021_01_age_ethnicity_region_QA'].groupby(['region', 'yr_id', 'name']).sum()
sum_vals = list(df.sum(axis=1))
row_sum_percentage = df.div(sum_vals, axis=0)*100
row_sum_percentage

In [172]:
row_sum_percentage.to_csv('C:/Users/cra/San Diego Association of Governments/SANDAG QA QC - Documents/Projects/2022/2022-56 Estimates 2021 QC/Data/2021_01/Proportion Checks/age_sex_ethnicity_row_wise_sums.csv')

In [129]:
df = new['2021_01_age_ethnicity_region_QA'].groupby(['region', 'yr_id', 'name']).sum()
sum_vals = list(df.sum(axis=0))
col_sum_percentage = df.div(sum_vals, axis=1)*100
col_sum_percentage

Hispanic  Non-Hispanic, White  \
region    yr_id name                                          
San Diego 2010  10 to 14      0.693788             0.372994   
                15 to 17      0.445666             0.244079   
                18 and 19     0.305812             0.203495   
                20 to 24      0.720757             0.654657   
                25 to 29      0.650553             0.624336   
...                                ...                  ...   
          2021  70 to 74      0.185246             0.499337   
                75 to 79      0.121653             0.336403   
                80 to 84      0.082648             0.219475   
                85 and Older  0.080606             0.248131   
                Under 5       0.624994             0.424063   

                              Non-Hispanic, Asian  \
region    yr_id name                                
San Diego 2010  10 to 14                 0.433983   
                15 to 17                 0.267522   
                18 and 19                0.239887   
                20 to 24                 0.705832   
                25 to 29                 0.672056   
...                                           ...   
          2021  70 to 74                 0.385580   
                75 to 79                 0.253040   
                80 to 84                 0.187060   
                85 and Older             0.178574   
                Under 5                  0.422643   

                              Non-Hispanic, Hawaiian or Pacific Islander  \
region    yr_id name                                                       
San Diego 2010  10 to 14                                        0.480541   
                15 to 17                                        0.339723   
                18 and 19                                       0.276942   
                20 to 24                                        0.844320   
                25 to 29                                        0.724625   
...                                                                  ...   
          2021  70 to 74                                        0.318014   
                75 to 79                                        0.160180   
                80 to 84                                        0.156660   
                85 and Older                                    0.144925   
                Under 5                                         0.406025   

                              Non-Hispanic, American Indian or Alaska Native  \
region    yr_id name                                                           
San Diego 2010  10 to 14                                            0.492421   
                15 to 17                                            0.342038   
                18 and 19                                           0.283241   
                20 to 24                                            0.845766   
                25 to 29                                            0.680118   
...                                                                      ...   
          2021  70 to 74                                            0.357868   
                75 to 79                                            0.196742   
                80 to 84                                            0.138511   
                85 and Older                                        0.128900   
                Under 5                                             0.533127   

                              Non-Hispanic, Other  \
region    yr_id name                                
San Diego 2010  10 to 14                 0.607473   
                15 to 17                 0.398799   
                18 and 19                0.258524   
                20 to 24                 0.712969   
                25 to 29                 0.841651   
...                                           ...   
          2021  70 to 74                 0.173895   
             

In [ ]:
# # Ethnicity by age code

# final = pd.DataFrame()


# for year in list(set(new['2021_01_age_ethnicity_region_QA'].groupby(['region', 'yr_id', 'name']).sum().index.get_level_values(1))):

#     df = new['2021_01_age_ethnicity_region_QA'].groupby(['region', 'yr_id', 'name']).sum().loc[("San Diego", year)]


#     sum_vals = list(df.sum(axis=0))
#     col_sum_percentage = df.div(sum_vals, axis=1)*100

#     df = col_sum_percentage


#     df['region'] = 'San Diego'
#     df['yr_id'] = year


#     df = df.reset_index()
#     final = final.append(df)


# # final.groupby(['region', 'yr_id', 'name']).sum().to_csv('C:/Users/cra/San Diego Association of Governments/SANDAG QA QC - Documents/Projects/2022/2022-56 Estimates 2021 QC/Data/2021_01/Proportion Checks/ethnicity_by_age_col_wise_sums.csv')

In [173]:
new['2021_01_age_sex_ethnicity_region_QA'].groupby(['region', 'yr_id', 'name', 'sex']).sum()

Hispanic  Non-Hispanic, White  \
region    yr_id name         sex                                     
San Diego 2010  10 to 14     Female     44161                33071   
                             Male       45878                35226   
                15 to 17     Female     27852                21611   
                             Male       29986                23081   
                18 and 19    Female     18981                16601   
...                                       ...                  ...   
          2021  80 to 84     Male        4092                17807   
                85 and Older Female      6828                28126   
                             Male        3633                17308   
                Under 5      Female     39979                37907   
                             Male       41132                39741   

                                     Non-Hispanic, Asian  \
region    yr_id name         sex                           
San Diego 2010  10 to 14     Female                 8634   
                             Male                   9316   
                15 to 17     Female                 5315   
                             Male                   5750   
                18 and 19    Female                 4888   
...                                                  ...   
          2021  80 to 84     Male                   3038   
                85 and Older Female                 4905   
                             Male                   2481   
                Under 5      Female                 8645   
                             Male                   8836   

                                     Non-Hispanic, Hawaiian or Pacific Islander  \
region    yr_id name         sex                                                  
San Diego 2010  10 to 14     Female                                         385   
                             Male                                           434   
                15 to 17     Female                                         270   
                             Male                                           309   
                18 and 19    Female                                         204   
...                                                                         ...   
          2021  80 to 84     Male                                           114   
                85 and Older Female                                         140   
                             Male                                           107   
                Under 5      Female                                         334   
                             Male                                           358   

                                     Non-Hispanic, American Indian or Alaska Native  \
region    yr_id name         sex                                                      
San Diego 2010  10 to 14     Female                                             398   
                             Male                                               473   
                15 to 17     Female                                             291   
                             Male                                               314   
                18 and 19    Female                                             216   
...                                                                             ...   
          2021  80 to 84     Male                                               118   
                85 and Older Female                                             134   
                             Male                                                94   
                Under 5      Female                                             425   
                             Male                                               518   

                                     Non-Hispanic, Other  \
region    yr_id name         sex                           
San Diego 201

In [174]:
new['2021_01_age_sex_ethnicity_region_QA'].groupby(['region', 'yr_id', 'name', 'sex']).sum().loc[("San Diego", year)]

Hispanic  Non-Hispanic, White  Non-Hispanic, Asian  \
name         sex                                                          
10 to 14     Female     44593                33337                 9185   
             Male       46629                35573                 9748   
15 to 17     Female     27717                22200                 5860   
             Male       30147                24607                 6424   
18 and 19    Female     20168                17700                 4817   
             Male       17331                25105                 4489   
20 to 24     Female     49977                42126                11369   
             Male       60881                56513                13408   
25 to 29     Female     38642                44723                12144   
             Male       41283                50485                12580   
30 to 34     Female     40324                51636                13877   
             Male       41464                57383                13200   
35 to 39     Female     37611                44525                14067   
             Male       36604                50299                12456   
40 to 44     Female     37897                43142                15104   
             Male       35021                49120                12554   
45 to 49     Female     34911                46986                13753   
             Male       32894                50933                11502   
5 to 9       Female     48667                36933                 9372   
             Male       50219                38836                 9823   
50 to 54     Female     30858                56253                12911   
             Male       28564                59718                10890   
55 to 59     Female     25214                59207                12411   
             Male       22148                60804                 9463   
60 and 61    Female      8213                23061                 4687   
             Male        6970                22531                 3662   
62 to 64     Female     10239                31941                 6477   
             Male        8635                31204                 4873   
65 to 69     Female     13568                48423                 9382   
             Male       10884                46416                 6487   
70 to 74     Female      9890                33805                 6313   
             Male        6832                31368                 4548   
75 to 79     Female      7544                25494                 5263   
             Male        4946                21574                 3551   
80 to 84     Female      5483                20497                 3890   
             Male        3348                16042                 2173   
85 and Older Female      5133                27904                 3285   
             Male        2887                16245                 1842   
Under 5      Female     49733                39338                 9238   
             Male       51666                41611                 9875   

                     Non-Hispanic, Hawaiian or Pacific Islander  \
name         sex                                                  
10 to 14     Female                                         369   
             Male                                           399   
15 to 17     Female                                         253   
             Male                                           341   
18 and 19    Female                                         198   
             Male                                           299   
20 to 24     Female                                         496   
             Male                                           688   
25 to 29     Female                                         535   
             Male                                           613   
30 to 34     Female                                         587   

In [176]:
# # Age Sex Ethnicity Code 

# final = pd.DataFrame()


# for year in list(set(new['2021_01_age_sex_ethnicity_region_QA'].groupby(['region', 'yr_id', 'name', 'sex']).sum().index.get_level_values(1))):

#     df = new['2021_01_age_sex_ethnicity_region_QA'].groupby(['region', 'yr_id', 'name', 'sex']).sum().loc[("San Diego", year)]


#     sum_vals = list(df.sum(axis=0))
#     col_sum_percentage = df.div(sum_vals, axis=1)*100

#     df = col_sum_percentage


#     df['region'] = 'San Diego'
#     df['yr_id'] = year


#     df = df.reset_index()
#     final = final.append(df)


# final.groupby(['region', 'yr_id', 'name', 'sex']).sum()


# final.groupby(['region', 'yr_id', 'name', 'sex']).sum().to_csv('C:/Users/cra/San Diego Association of Governments/SANDAG QA QC - Documents/Projects/2022/2022-56 Estimates 2021 QC/Data/2021_01/Proportion Checks/age_sex_ethnicity_col_wise_sums.csv')

In [130]:
col_sum_percentage.to_csv('C:/Users/cra/San Diego Association of Governments/SANDAG QA QC - Documents/Projects/2022/2022-56 Estimates 2021 QC/Data/2021_01/Proportion Checks/ethnicity_by_age_col_wise_sums.csv')

### DOF Proportion Checks 

In [77]:
# Download the data
dof_data = pd.read_csv('C:/Users/cra/San Diego Association of Governments/SANDAG QA QC - Documents/Projects/2022/2022-56 Estimates 2021 QC/Data/2021_01/DOF_Data/DOF_2021_07_14_age_sex_ethnicity_region_QA.csv')

In [78]:
list(set(new['2021_01_age_sex_ethnicity_region_QA'].groupby(['region', 'yr_id', 'name', 'sex']).sum().index.get_level_values(1)))

[2016, 2017, 2018, 2019, 2020, 2021, 2010, 2011, 2012, 2013, 2014, 2015]

In [81]:
dof_data = dof_data[dof_data['yr_id'].isin(list(set(new['2021_01_age_sex_ethnicity_region_QA'].groupby(['region', 'yr_id', 'name', 'sex']).sum().index.get_level_values(1))))]

In [82]:

df = dof_data.groupby(['region', 'yr_id', 'name', 'sex']).sum()
sum_vals = list(df.sum(axis=1))
row_sum_percentage = df.div(sum_vals, axis=0)*100
row_sum_percentage

Hispanic  \
region    yr_id name         sex              
San Diego 2010  10 to 14     F    45.611643   
                             M    45.027930   
                15 to 17     F    40.949157   
                             M    40.751195   
                18 and 19    F    39.227606   
...                                     ...   
          2021  80 to 84     M    15.382061   
                85 and Older F    16.611950   
                             M    14.880976   
                Under 5      F    40.810502   
                             M    40.443574   

                                  Non-Hispanic, American Indian or Alaska Native  \
region    yr_id name         sex                                                   
San Diego 2010  10 to 14     F                                          0.421431   
                             M                                          0.491093   
                15 to 17     F                                          0.502052   
                             M                                          0.476516   
                18 and 19    F                                          0.556499   
...                                                                          ...   
          2021  80 to 84     M                                          0.457514   
                85 and Older F                                          0.290779   
                             M                                          0.348556   
                Under 5      F                                          0.457507   
                             M                                          0.524893   

                                  Non-Hispanic, Asian  Non-Hispanic, Black  \
region    yr_id name         sex                                             
San Diego 2010  10 to 14     F               9.048372             5.017921   
                             M               9.260328             5.016582   
                15 to 17     F              10.422084             4.960016   
                             M              10.066014             5.287199   
                18 and 19    F              10.763454             5.354908   
...                                               ...                  ...   
          2021  80 to 84     M              11.733018             3.468251   
                85 and Older F              11.971203             2.724885   
                             M              10.135858             3.045906   
                Under 5      F               9.009538             4.916106   
                             M               8.896334             4.939757   

                                  Non-Hispanic, Hawaiian or Pacific Islander  \
region    yr_id name         sex                                               
San Diego 2010  10 to 14     F                                      0.393542   
                             M                                      0.422614   
                15 to 17     F                                      0.485699   
                             M                                      0.496244   
                18 and 19    F                                      0.433578   
...                                                                      ...   
          2021  80 to 84     M                                      0.398480   
                85 and Older F                                      0.342369   
                             M                                      0.443617   
                Under 5      F                                      0.351525   
                             M                                      0.358875   

                                  Non-Hispanic, Two or More Races  \
region    yr_id name         sex                                    
San Diego 2010  10 to 14     F                           5.174925   
                             M                           5.036147  

In [83]:
row_sum_percentage.to_csv('C:/Users/cra/San Diego Association of Governments/SANDAG QA QC - Documents/Projects/2022/2022-56 Estimates 2021 QC/Data/2021_01/DOF_Data/DOF_age_sex_ethnicity_row_wise_sums.csv')

In [5]:
dof_data

,region,yr_id,name,sex,Hispanic,"Non-Hispanic, American Indian or Alaska Native","Non-Hispanic, Asian","Non-Hispanic, Black","Non-Hispanic, Hawaiian or Pacific Islander","Non-Hispanic, Two or More Races","Non-Hispanic, White"
0,San Diego,2010,Under 5,F,46416,440,8635,4754,386,6377,33850
1,San Diego,2010,Under 5,M,48044,465,8973,4921,384,6616,35747
2,San Diego,2010,5 to 9,F,43723,418,8829,4376,378,5596,31994
3,San Diego,2010,5 to 9,M,45339,438,9209,4673,384,5924,33958
4,San Diego,2010,10 to 14,F,44158,408,8760,4858,381,5010,33238
...,...,...,...,...,...,...,...,...,...,...,...
2035,San Diego,2060,75 to 79,M,28389,377,9691,4395,381,2512,42073
2036,San Diego,2060,80 to 84,F,28171,328,11364,3540,379,2197,36842
2037,San Diego,2060,80 to 84,M,23064,320,8487,3388,366,1930,35191
2038,San Diego,2060,85 and Older,F,48066,581,20808,6279,708,2970,69277


In [11]:
# Age Sex Ethnicity Code 

final = pd.DataFrame()


for year in list(set(new['2021_01_age_sex_ethnicity_region_QA'].groupby(['region', 'yr_id', 'name', 'sex']).sum().index.get_level_values(1))):

    df = dof_data.groupby(['region', 'yr_id', 'name', 'sex']).sum().loc[("San Diego", year)]


    sum_vals = list(df.sum(axis=0))
    col_sum_percentage = df.div(sum_vals, axis=1)*100

    df = col_sum_percentage


    df['region'] = 'San Diego'
    df['yr_id'] = year


    df = df.reset_index()
    final = final.append(df)


final.groupby(['region', 'yr_id', 'name', 'sex']).sum()


# final.groupby(['region', 'yr_id', 'name', 'sex']).sum().to_csv('C:/Users/cra/San Diego Association of Governments/SANDAG QA QC - Documents/Projects/2022/2022-56 Estimates 2021 QC/Data/2021_01/DOF_Data/DOF_age_sex_ethnicity_col_wise_sums.csv')

# Department of Finance Checks 
There's 2 DOF sources that the estimates control to. The 2021 vintage DOF Population and Housing Estimates E5 that's in [socioec_data].[ca_dof].[population_housing_estimates]. This gives household, group quarter data jurisdiction level. Then there's the 7.14.2021 DOF Projections that gives single year age group and ethnicity characteristics. It's also in the socioec_data database I just don't have the name of the table at the moment since I can't remote access my computer since the power outage.

### 2021 Vintage DOF Population and Housing Estimates E5
[socioec_data].[ca_dof].[population_housing_estimates]


- Which year do I compare the DOF numbers to? The base year?

In [21]:
conn = pyodbc.connect('Driver={ODBC Driver 17 for SQL Server};'
                    'Server=DDAMWSQL16.sandag.org;'
                    'Database=socioec_data;'
                    'Trusted_Connection=yes;')

In [73]:
dof_pop_query = '''SELECT *
  FROM [socioec_data].[ca_dof].[population_housing_estimates]
  WHERE area_name = 'San Diego' AND summary_type='Total' AND area_type = 'county' AND vintage_yr = 2021;'''

dof_pop =  pd.read_sql_query(dof_pop_query, conn) #example df

dof_pop

,area_type,area_name,summary_type,county_name,est_yr,est_md,total_pop,household_pop,group_quarters,total_hu,single,single_detached,single_attached,multiple,two_to_four,five_plus,mobile_homes,occupied,unoccupied,vintage_yr
0,County,San Diego,Total,San Diego,2017,JAN_1,3303366,3197067,106299,1201517,721664,616128,105536,433837,86281,347556,46016,1129464,72053,2021
1,County,San Diego,Total,San Diego,2010,JAN_1,3095313,2991515,103798,1164028,707604,603441,104163,410658,84621,326037,45766,1086113,77915,2021
2,County,San Diego,Total,San Diego,2014,JAN_1,3232762,3127529,105233,1180875,715522,610783,104739,419449,85494,333955,45904,1107562,73313,2021
3,County,San Diego,Total,San Diego,2020,JAN_1,3331279,3218869,112410,1226879,729293,622861,106432,451206,87814,363392,46380,1153981,72898,2021
4,County,San Diego,Total,San Diego,2011,JAN_1,3127603,3025703,101900,1166858,709313,605022,104291,411748,84620,327128,45797,1089420,77438,2021
5,County,San Diego,Total,San Diego,2018,JAN_1,3321118,3210345,110773,1210138,723430,617674,105756,440623,86682,353941,46085,1137791,72347,2021
6,County,San Diego,Total,San Diego,2015,JAN_1,3264706,3156628,108078,1188445,717461,612517,104944,425048,85792,339256,45936,1114909,73536,2021
7,County,San Diego,Total,San Diego,2021,JAN_1,3315404,3219058,96346,1234087,730771,624045,106726,456850,89426,367424,46466,1160772,73315,2021
8,County,San Diego,Total,San Diego,2012,JAN_1,3161808,3058316,103492,1170267,711359,606978,104381,413069,84884,328185,45839,1093511,76756,2021
9,County,San Diego,Total,San Diego,2016,JAN_1,3283009,3175539,107470,1193395,719600,614416,105184,427835,85921,341914,45960,1121242,72153,2021


In [ ]:
'''
- Create a table of differences (see below)
- Post it to Mike to see if identical values are strange 


'''

total_hu, 'units'
- add vacancy and unocupied

In [49]:
new['2021_01_consolidated_region_QA'][new['2021_01_consolidated_region_QA']['yr_id'] == 2021][['Total Population','Household Population', 'households', 'Group Quarters - Military',
       'Group Quarters - College', 'Group Quarters - Other', 'Single Family - Detached',
       'Single Family - Multiple Unit', 'Multifamily', 'Mobile Home',
       'Single-family Detached', 'Single-family Attached']].reset_index(drop=True).columns

Index(['Total Population', 'Household Population', 'households',
       'Group Quarters - Military', 'Group Quarters - College',
       'Group Quarters - Other', 'Single Family - Detached',
       'Single Family - Multiple Unit', 'Multifamily', 'Mobile Home',
       'Single-family Detached', 'Single-family Attached'],
      dtype='object')

In [80]:
new['2021_01_consolidated_region_QA'].columns

Index(['region', 'yr_id', 'Under 5', '5 to 9', '10 to 14', '15 to 17',
       '18 and 19', '20 to 24', '25 to 29', '30 to 34', '35 to 39', '40 to 44',
       '45 to 49', '50 to 54', '55 to 59', '60 and 61', '62 to 64', '65 to 69',
       '70 to 74', '75 to 79', '80 to 84', '85 and Older', 'Hispanic',
       'Non-Hispanic, White', 'Non-Hispanic, Black',
       'Non-Hispanic, American Indian or Alaska Native', 'Non-Hispanic, Asian',
       'Non-Hispanic, Hawaiian or Pacific Islander', 'Non-Hispanic, Other',
       'Non-Hispanic, Two or More Races', 'Less than $15,000',
       '$15,000 to $29,999', '$30,000 to $44,999', '$45,000 to $59,999',
       '$60,000 to $74,999', '$75,000 to $99,999', '$100,000 to $124,999',
       '$125,000 to $149,999', '$150,000 to $199,999', '$200,000 or more',
       'households', 'Single Family - Detached',
       'Single Family - Multiple Unit', 'Multifamily', 'Mobile Home',
       'Single-family Detached', 'Single-family Attached', 'units',
       'unoccupi

In [65]:
new['2021_01_consolidated_region_QA'].columns

Index(['region', 'yr_id', 'Under 5', '5 to 9', '10 to 14', '15 to 17',
       '18 and 19', '20 to 24', '25 to 29', '30 to 34', '35 to 39', '40 to 44',
       '45 to 49', '50 to 54', '55 to 59', '60 and 61', '62 to 64', '65 to 69',
       '70 to 74', '75 to 79', '80 to 84', '85 and Older', 'Hispanic',
       'Non-Hispanic, White', 'Non-Hispanic, Black',
       'Non-Hispanic, American Indian or Alaska Native', 'Non-Hispanic, Asian',
       'Non-Hispanic, Hawaiian or Pacific Islander', 'Non-Hispanic, Other',
       'Non-Hispanic, Two or More Races', 'Less than $15,000',
       '$15,000 to $29,999', '$30,000 to $44,999', '$45,000 to $59,999',
       '$60,000 to $74,999', '$75,000 to $99,999', '$100,000 to $124,999',
       '$125,000 to $149,999', '$150,000 to $199,999', '$200,000 or more',
       'households', 'Single Family - Detached',
       'Single Family - Multiple Unit', 'Multifamily', 'Mobile Home',
       'Single-family Detached', 'Single-family Attached', 'units',
       'unoccupi

In [29]:
dof_pop[dof_pop['est_yr'] == 2021]['total_pop']

7    3315404
Name: total_pop, dtype: int64

In [67]:
list(new['2021_01_consolidated_region_QA']['yr_id'])

[2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021]

In [75]:
# DOF_1_check = pd.DataFrame()

# temp_df = pd.DataFrame()

# for year in list(new['2021_01_consolidated_region_QA']['yr_id']):
#     # Data Setup 
#     dof_df = dof_pop[dof_pop['est_yr'] == year]
#     estimates_dof = new['2021_01_consolidated_region_QA'][new['2021_01_consolidated_region_QA']['yr_id'] == year]

#     # Population Commparison
#     dof_total_population_value =  int(dof_df['total_pop']) 
#     estimates_total_population_value = int(estimates_dof['Total Population'])
#     if estimates_total_population_value == 0:
#         temp_df['Household Population  Diff'] = [f"Zero in denominater, actual diff is: {dof_total_population_value-estimates_total_population_value}"]
#     else:
#         temp_df['Total Population Diff'] = [dof_total_population_value-estimates_total_population_value]

#     # Household Population Comparison 
#     dof_household_population_value =  int(dof_df['household_pop']) 
#     estimates_household_population_value = int(estimates_dof['Household Population'])
#     if estimates_household_population_value == 0:
#         temp_df['Household Population  Diff'] = [f"Zero in denominater, actual diff is: {dof_household_population_value-estimates_household_population_value}"]
#     else: 
#         temp_df['Household Population  Diff'] = [dof_household_population_value-estimates_household_population_value]

#     # Group Quarters Comparison 
#     dof_group_quarters_value =  int(dof_df['group_quarters']) 
#     estimates_group_quarters_value = int(estimates_dof['Group Quarters - Military']) + int(estimates_dof['Group Quarters - College']) + int(estimates_dof['Group Quarters - Other'])
#     if estimates_group_quarters_value == 0:
#         temp_df['Group Quarters Diff'] = [f"Zero in denominater, actual diff is: {dof_group_quarters_value-estimates_group_quarters_value}"]
#     else: 
#         temp_df['Group Quarters Diff'] = [dof_group_quarters_value-estimates_group_quarters_value]

#     # Single Family Detached Comparison 
#     dof_single_detached_value =  int(dof_df['single_detached']) 
#     estimates_single_detached_value = int(estimates_dof['Single Family - Detached'])
#     if estimates_single_detached_value == 0:
#         temp_df['Single Detached Diff'] = [f"Zero in denominater, actual diff is: {dof_single_detached_value-estimates_single_detached_value}"]
#     else: 
#         temp_df['Single Detached  Diff'] = [dof_single_detached_value-estimates_single_detached_value]

#     # Single Family Attchaded Comparison 
#     dof_single_attached_value =  int(dof_df['single_attached']) 
#     estimates_single_attached_value = int(estimates_dof['Single-family Attached'])
#     if estimates_single_attached_value == 0:
#         temp_df['Single Attached  Diff'] = [f"Zero in denominater, actual diff is: {dof_single_attached_value-estimates_single_attached_value}"]
#     else:
#         temp_df['Single Attached Diff'] = [dof_single_attached_value-estimates_single_attached_value]

#     # Setup the Year 
#     temp_df['Year'] = year

#     DOF_1_check = DOF_1_check.append(temp_df)
# DOF_1_check.set_index('Year')

,Total Population Diff,Household Population Diff,Group Quarters Diff,Single Detached Diff,Single Attached Diff
Year,,,,,
2010,-1,-1833,1832,43180,"Zero in denominater, actual diff is: 104163"
2011,-1,-1,0,43776,"Zero in denominater, actual diff is: 104291"
2012,-1,-1,0,44132,"Zero in denominater, actual diff is: 104381"
2013,-1,-1,0,42999,"Zero in denominater, actual diff is: 104473"
2014,-1,-1,0,42852,"Zero in denominater, actual diff is: 104739"
2015,-1,-1,0,42383,"Zero in denominater, actual diff is: 104944"
2016,-1,-1,0,41937,"Zero in denominater, actual diff is: 105184"
2017,-1,-1,0,40895,"Zero in denominater, actual diff is: 105536"
2018,0,0,0,39642,"Zero in denominater, actual diff is: 105756"


In [85]:
DOF_1_check = pd.DataFrame()

temp_df = pd.DataFrame()

for year in list(new['2021_01_consolidated_region_QA']['yr_id']):
    # Data Setup 
    dof_df = dof_pop[dof_pop['est_yr'] == year]
    estimates_dof = new['2021_01_consolidated_region_QA'][new['2021_01_consolidated_region_QA']['yr_id'] == year]

    # Population Commparison
    dof_total_population_value =  int(dof_df['total_pop']) 
    estimates_total_population_value = int(estimates_dof['Total Population'])
    if estimates_total_population_value == 0:
        temp_df['Household Population Diff'] = [f"Zero in denominater, actual diff is: {dof_total_population_value-estimates_total_population_value}"]
    else:
        temp_df['Total Population Diff'] = [dof_total_population_value-estimates_total_population_value]

    # Household Population Comparison 
    dof_household_population_value =  int(dof_df['household_pop']) 
    estimates_household_population_value = int(estimates_dof['Household Population'])
    if estimates_household_population_value == 0:
        temp_df['Household Population Diff'] = [f"Zero in denominater, actual diff is: {dof_household_population_value-estimates_household_population_value}"]
    else: 
        temp_df['Household Population Diff'] = [dof_household_population_value-estimates_household_population_value]

    # Group Quarters Comparison 
    dof_group_quarters_value =  int(dof_df['group_quarters']) 
    estimates_group_quarters_value = int(estimates_dof['Group Quarters - Military']) + int(estimates_dof['Group Quarters - College']) + int(estimates_dof['Group Quarters - Other'])
    if estimates_group_quarters_value == 0:
        temp_df['Group Quarters Diff'] = [f"Zero in denominater, actual diff is: {dof_group_quarters_value-estimates_group_quarters_value}"]
    else: 
        temp_df['Group Quarters Diff'] = [dof_group_quarters_value-estimates_group_quarters_value]

    # # Single Family Detached Comparison 
    # dof_single_detached_value =  int(dof_df['single_detached']) 
    # estimates_single_detached_value = int(estimates_dof['Single Family - Detached'])
    # temp_df['Single Detached Diff'] = [dof_single_detached_value-estimates_single_detached_value]

    # # Single Family Attchaded Comparison 
    # dof_single_attached_value =  int(dof_df['single_attached']) 
    # estimates_single_attached_value = int(estimates_dof['Single-family Attached'])
    # if estimates_single_attached_value == 0:
    #     temp_df['Single Attached Diff'] = [dof_single_attached_value-estimates_single_attached_value]
    # else:
    #     temp_df['Single Attached Diff'] = [dof_single_attached_value-estimates_single_attached_value]
    
    # Total Housing Units Comparison 
    dof_total_housing_value =  int(dof_df['total_hu']) 
    estimates_total_housing_value = int(estimates_dof['units'])
    temp_df['Total Housing Diff'] = [dof_total_housing_value-estimates_total_housing_value]

    # Total Vacancy Comparison 
    dof_vacancy_value =  int(dof_df['unoccupied']) 
    estimates_vacancy_value = int(estimates_dof['vacancy'])
    temp_df['Vacancy Diff'] = [dof_vacancy_value-estimates_vacancy_value]


    # Setup the Year 
    temp_df['Year'] = year

    DOF_1_check = DOF_1_check.append(temp_df)
DOF_1_check.set_index('Year')

,Total Population Diff,Household Population Diff,Group Quarters Diff,Total Housing Diff,Vacancy Diff
Year,,,,,
2010,-1,-1833,1832,9026,9779
2011,-1,-1,0,8434,8435
2012,-1,-1,0,7935,7936
2013,-1,-1,0,9363,9364
2014,-1,-1,0,8841,8842
2015,-1,-1,0,9588,9589
2016,-1,-1,0,10412,10413
2017,-1,-1,0,10962,10963
2018,0,0,0,12429,12429


In [61]:
int(estimates_dof['Single-family Attached'])

0

In [53]:
dof_group_quarters_value

96346

In [55]:
estimates_group_quarters_value

96346

In [34]:
estimates_total_population_value

11    3315404
Name: Total Population, dtype: int64

In [58]:
# The total population numbers for 2021 are identical 
(3315404-3315404) / 3315404

0.0

### Second DOF Data

In [68]:
dof_eth_query = '''SELECT [county_fips_code]
      ,[fiscal_yr]
      ,[race_code]
      ,[sex]
      ,[age]
      ,[population]
  FROM [socioec_data].[ca_dof].[population_proj_2021_07_14]
  WHERE county_fips_code = '06073';'''

dof_eth =  pd.read_sql_query(dof_eth_query, conn) #example df
dof_eth

,county_fips_code,fiscal_yr,race_code,sex,age,population
0,6073,2029,6,M,109,0
1,6073,2029,6,M,110,0
2,6073,2029,7,F,0,6838
3,6073,2029,7,F,1,6687
4,6073,2029,7,F,2,6991
...,...,...,...,...,...,...
79249,6073,2060,7,M,106,7
79250,6073,2060,7,M,107,1
79251,6073,2060,7,M,108,1
79252,6073,2060,7,M,109,1


In [69]:
dof_eth.columns

Index(['county_fips_code', 'fiscal_yr', 'race_code', 'sex', 'age',
       'population'],
      dtype='object')

In [70]:
dof_eth[['fiscal_yr', 'race_code', 'sex', 'age','population']].groupby(['fiscal_yr', 'race_code', 'sex', 'age']).sum()

population
fiscal_yr race_code sex age            
2010      1         F   0          6877
                        1          6731
                        2          6743
                        3          6848
                        4          6651
...                                 ...
2060      7         M   106           7
                        107           1
                        108           1
                        109           1
                        110           0

[79254 rows x 1 columns]